In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train_zip.csv")

In [3]:
df.head()

,Unnamed: 0,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,...,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn
0,1812523,0,0x39CE9F19213F5D498775ABA329A944CD,0xB492A0893AF5C042A81AB98FFBF96B1D,2,1,2015,2015Q2,-1,41,...,-1,-1,-1,398017.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1,0
1,1940300,1,0xE886F2AC4042F947B94774E65DFC49E1,0x30405A5ED70D134DBACEA479F4CA1E4A,1,2,2015,2015Q4,-1,39,...,1,-1,-1,141006.0,МОСКОВСКАЯ ОБЛ,1,-1,0,0,0
2,3110992,2,0xB8610F684555D14E9D0819F03F69338B,0x810C005056AB57EC11EBA287AC552DD7,2,1,2022,2022Q2,1,47,...,1,-1,-1,162626.0,ВОЛОГОДСКАЯ ОБЛ,1,1,0,1,0
3,2855815,0,0xECA4660A88958844BC707D88DD13FB44,0x3225ED4DFFCD4042BB36FE353B42ACBD,1,2,2020,2020Q4,1,48,...,-1,-1,-1,398036.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,0,0
4,2778810,2,0x6FB0DAB05D8EC24DABD333F2DC415270,0x58998F4E17293D49953B9943F2B794E3,1,2,2012,2012Q4,1,38,...,-1,-1,-1,162609.0,ВОЛОГОДСКАЯ ОБЛ,1,1,0,0,0


In [31]:
nmbr = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_nmbr_per_year' : 'sum'}).reset_index()

In [30]:
s = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_sum_per_year' : 'sum'}).reset_index()

In [7]:
number_in_scientific_notation = 5.158350e+03
number_in_decimal = "{:.9f}".format(number_in_scientific_notation)

print(number_in_decimal)  

5158.350000000


In [36]:
churn = df.query('year > 2012 and year < 2023 and churn == 0').groupby('year').agg({'churn' : 'count'}).reset_index()

In [29]:
all = df.query('year > 2012 and year < 2023').groupby('year').agg({'churn' : 'count'}).reset_index()

In [37]:
new = nmbr.merge(s.merge(all.merge(churn, on='year'), on='year') , on='year')

In [39]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [41]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [44]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [46]:
new.loyl_din = new.loyl_din.fillna(0)

In [48]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [55]:
new.rename(columns=({'year' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [58]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [59]:
new[new_order]

,name,loyl_din,pmnts_nmbr_din,pmnts_sum_din,loyl,pmnts_nmbr,pmnts_sum
0,2013,0.000000,0.000000,0.000000,97.665370,39263,5.186857e+07
1,2014,0.411879,-11.738787,19.783551,98.067633,34654,6.213001e+07
2,2015,0.258531,8.902291,13.574433,98.321168,37739,7.056381e+07
3,2016,-0.078743,11.004531,38.083161,98.243747,41892,9.743674e+07
4,2017,-0.304700,18.793564,37.020180,97.944398,49765,1.335080e+08
5,2018,0.542860,10.396865,18.848835,98.476099,54939,1.586727e+08
6,2019,0.808372,3.094341,20.206625,99.272152,56639,1.907351e+08
7,2020,-0.855711,-1.221773,15.148304,98.422669,55947,2.196282e+08
8,2021,-0.185400,10.472411,6.743915,98.240193,61806,2.344398e+08
9,2022,0.830211,8.751578,12.400092,99.055794,67215,2.635105e+08


In [60]:
new[new_order].T.to_json()

'{"0":{"name":2013.0,"loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":97.6653696498,"pmnts_nmbr":39263.0,"pmnts_sum":51868567.5700000003},"1":{"name":2014.0,"loyl_din":0.4118790538,"pmnts_nmbr_din":-11.7387871533,"pmnts_sum_din":19.7835505794,"loyl":98.0676328502,"pmnts_nmbr":34654.0,"pmnts_sum":62130011.8699999973},"2":{"name":2015.0,"loyl_din":0.2585307972,"pmnts_nmbr_din":8.9022912218,"pmnts_sum_din":13.5744328645,"loyl":98.3211678832,"pmnts_nmbr":37739.0,"pmnts_sum":70563808.6200000048},"3":{"name":2016.0,"loyl_din":-0.0787431752,"pmnts_nmbr_din":11.0045311217,"pmnts_sum_din":38.0831608236,"loyl":98.2437466738,"pmnts_nmbr":41892.0,"pmnts_sum":97436737.3400000036},"4":{"name":2017.0,"loyl_din":-0.3047003426,"pmnts_nmbr_din":18.7935644037,"pmnts_sum_din":37.020179549,"loyl":97.9443976411,"pmnts_nmbr":49765.0,"pmnts_sum":133507992.450000003},"5":{"name":2018.0,"loyl_din":0.5428602193,"pmnts_nmbr_din":10.3968652668,"pmnts_sum_din":18.8488350159,"loyl":98.476098813,"pmnts_

In [ ]:
nmbr = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_nmbr_per_year' : 'sum'}).reset_index()

In [ ]:
s = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_sum_per_year' : 'sum'}).reset_index()

In [ ]:
number_in_scientific_notation = 5.158350e+03
number_in_decimal = "{:.9f}".format(number_in_scientific_notation)

print(number_in_decimal)  

5158.350000000


In [ ]:
churn = df.query('year > 2012 and year < 2023 and churn == 0').groupby('year').agg({'churn' : 'count'}).reset_index()

In [ ]:
all = df.query('year > 2012 and year < 2023').groupby('year').agg({'churn' : 'count'}).reset_index()

In [ ]:
new = nmbr.merge(s.merge(all.merge(churn, on='year'), on='year') , on='year')

In [ ]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [ ]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [ ]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [ ]:
new.loyl_din = new.loyl_din.fillna(0)

In [ ]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [ ]:
new.rename(columns=({'year' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [ ]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [ ]:
new[new_order]

,name,loyl_din,pmnts_nmbr_din,pmnts_sum_din,loyl,pmnts_nmbr,pmnts_sum
0,2013,0.000000,0.000000,0.000000,97.665370,39263,5.186857e+07
1,2014,0.411879,-11.738787,19.783551,98.067633,34654,6.213001e+07
2,2015,0.258531,8.902291,13.574433,98.321168,37739,7.056381e+07
3,2016,-0.078743,11.004531,38.083161,98.243747,41892,9.743674e+07
4,2017,-0.304700,18.793564,37.020180,97.944398,49765,1.335080e+08
5,2018,0.542860,10.396865,18.848835,98.476099,54939,1.586727e+08
6,2019,0.808372,3.094341,20.206625,99.272152,56639,1.907351e+08
7,2020,-0.855711,-1.221773,15.148304,98.422669,55947,2.196282e+08
8,2021,-0.185400,10.472411,6.743915,98.240193,61806,2.344398e+08
9,2022,0.830211,8.751578,12.400092,99.055794,67215,2.635105e+08


In [ ]:
new[new_order].T.to_json()

'{"0":{"name":2013.0,"loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":97.6653696498,"pmnts_nmbr":39263.0,"pmnts_sum":51868567.5700000003},"1":{"name":2014.0,"loyl_din":0.4118790538,"pmnts_nmbr_din":-11.7387871533,"pmnts_sum_din":19.7835505794,"loyl":98.0676328502,"pmnts_nmbr":34654.0,"pmnts_sum":62130011.8699999973},"2":{"name":2015.0,"loyl_din":0.2585307972,"pmnts_nmbr_din":8.9022912218,"pmnts_sum_din":13.5744328645,"loyl":98.3211678832,"pmnts_nmbr":37739.0,"pmnts_sum":70563808.6200000048},"3":{"name":2016.0,"loyl_din":-0.0787431752,"pmnts_nmbr_din":11.0045311217,"pmnts_sum_din":38.0831608236,"loyl":98.2437466738,"pmnts_nmbr":41892.0,"pmnts_sum":97436737.3400000036},"4":{"name":2017.0,"loyl_din":-0.3047003426,"pmnts_nmbr_din":18.7935644037,"pmnts_sum_din":37.020179549,"loyl":97.9443976411,"pmnts_nmbr":49765.0,"pmnts_sum":133507992.450000003},"5":{"name":2018.0,"loyl_din":0.5428602193,"pmnts_nmbr_din":10.3968652668,"pmnts_sum_din":18.8488350159,"loyl":98.476098813,"pmnts_

In [ ]:
nmbr = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_nmbr_per_year' : 'sum'}).reset_index()

In [ ]:
s = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_sum_per_year' : 'sum'}).reset_index()

In [ ]:
number_in_scientific_notation = 5.158350e+03
number_in_decimal = "{:.9f}".format(number_in_scientific_notation)

print(number_in_decimal)  

5158.350000000


In [ ]:
churn = df.query('year > 2012 and year < 2023 and churn == 0').groupby('year').agg({'churn' : 'count'}).reset_index()

In [ ]:
all = df.query('year > 2012 and year < 2023').groupby('year').agg({'churn' : 'count'}).reset_index()

In [ ]:
new = nmbr.merge(s.merge(all.merge(churn, on='year'), on='year') , on='year')

In [ ]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [ ]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [ ]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [ ]:
new.loyl_din = new.loyl_din.fillna(0)

In [ ]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [ ]:
new.rename(columns=({'year' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [ ]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [ ]:
new[new_order]

,name,loyl_din,pmnts_nmbr_din,pmnts_sum_din,loyl,pmnts_nmbr,pmnts_sum
0,2013,0.000000,0.000000,0.000000,97.665370,39263,5.186857e+07
1,2014,0.411879,-11.738787,19.783551,98.067633,34654,6.213001e+07
2,2015,0.258531,8.902291,13.574433,98.321168,37739,7.056381e+07
3,2016,-0.078743,11.004531,38.083161,98.243747,41892,9.743674e+07
4,2017,-0.304700,18.793564,37.020180,97.944398,49765,1.335080e+08
5,2018,0.542860,10.396865,18.848835,98.476099,54939,1.586727e+08
6,2019,0.808372,3.094341,20.206625,99.272152,56639,1.907351e+08
7,2020,-0.855711,-1.221773,15.148304,98.422669,55947,2.196282e+08
8,2021,-0.185400,10.472411,6.743915,98.240193,61806,2.344398e+08
9,2022,0.830211,8.751578,12.400092,99.055794,67215,2.635105e+08


In [ ]:
new[new_order].T.to_json()

'{"0":{"name":2013.0,"loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":97.6653696498,"pmnts_nmbr":39263.0,"pmnts_sum":51868567.5700000003},"1":{"name":2014.0,"loyl_din":0.4118790538,"pmnts_nmbr_din":-11.7387871533,"pmnts_sum_din":19.7835505794,"loyl":98.0676328502,"pmnts_nmbr":34654.0,"pmnts_sum":62130011.8699999973},"2":{"name":2015.0,"loyl_din":0.2585307972,"pmnts_nmbr_din":8.9022912218,"pmnts_sum_din":13.5744328645,"loyl":98.3211678832,"pmnts_nmbr":37739.0,"pmnts_sum":70563808.6200000048},"3":{"name":2016.0,"loyl_din":-0.0787431752,"pmnts_nmbr_din":11.0045311217,"pmnts_sum_din":38.0831608236,"loyl":98.2437466738,"pmnts_nmbr":41892.0,"pmnts_sum":97436737.3400000036},"4":{"name":2017.0,"loyl_din":-0.3047003426,"pmnts_nmbr_din":18.7935644037,"pmnts_sum_din":37.020179549,"loyl":97.9443976411,"pmnts_nmbr":49765.0,"pmnts_sum":133507992.450000003},"5":{"name":2018.0,"loyl_din":0.5428602193,"pmnts_nmbr_din":10.3968652668,"pmnts_sum_din":18.8488350159,"loyl":98.476098813,"pmnts_

In [18]:
nmbr = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_nmbr_per_year' : 'mean'}).reset_index()

In [19]:
s = df.query('year > 2012 and year < 2023').groupby('year').agg({'pmnts_sum_per_year' : 'mean'}).reset_index()

In [ ]:
number_in_scientific_notation = 5.158350e+03
number_in_decimal = "{:.9f}".format(number_in_scientific_notation)

print(number_in_decimal)  

5158.350000000


In [20]:
churn = df.query('year > 2012 and year < 2023 and churn == 0').groupby('year').agg({'churn' : 'count'}).reset_index()

In [21]:
all = df.query('year > 2012 and year < 2023').groupby('year').agg({'churn' : 'count'}).reset_index()

In [22]:
new = nmbr.merge(s.merge(all.merge(churn, on='year'), on='year') , on='year')

In [23]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [24]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [25]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [26]:
new.loyl_din = new.loyl_din.fillna(0)

In [27]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [28]:
new.rename(columns=({'year' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [29]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [30]:
new[new_order]

,name,loyl_din,pmnts_nmbr_din,pmnts_sum_din,loyl,pmnts_nmbr,pmnts_sum
0,2013,0.000000,0.000000,0.000000,97.665370,7.638716,10091.161006
1,2014,0.411879,-4.712742,29.318935,98.067633,7.278723,13049.781951
2,2015,0.258531,-5.386166,-1.327030,98.321168,6.886679,12876.607412
3,2016,-0.078743,7.912867,34.237311,98.243747,7.431613,17285.211520
4,2017,-0.304700,12.828866,30.140312,97.944398,8.385004,22495.028214
5,2018,0.542860,5.101924,13.148514,98.476099,8.812801,25452.790132
6,2019,0.808372,1.691475,18.570902,99.272152,8.961867,30179.602821
7,2020,-0.855711,-12.858962,1.582535,98.422669,7.809464,30657.205684
8,2021,-0.185400,6.317082,2.728830,98.240193,8.302794,31493.788686
9,2022,0.830211,38.977982,43.640564,99.055794,11.539056,45237.855566


In [31]:
new[new_order].T.to_json()

'{"0":{"name":2013.0,"loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":97.6653696498,"pmnts_nmbr":7.6387159533,"pmnts_sum":10091.1610058366},"1":{"name":2014.0,"loyl_din":0.4118790538,"pmnts_nmbr_din":-4.7127422743,"pmnts_sum_din":29.318935093,"loyl":98.0676328502,"pmnts_nmbr":7.2787229574,"pmnts_sum":13049.7819512707},"2":{"name":2015.0,"loyl_din":0.2585307972,"pmnts_nmbr_din":-5.3861663308,"pmnts_sum_din":-1.3270301336,"loyl":98.3211678832,"pmnts_nmbr":6.8866788321,"pmnts_sum":12876.6074124088},"3":{"name":2016.0,"loyl_din":-0.0787431752,"pmnts_nmbr_din":7.9128668701,"pmnts_sum_din":34.2373108592,"loyl":98.2437466738,"pmnts_nmbr":7.4316125599,"pmnts_sum":17285.2115203122},"4":{"name":2017.0,"loyl_din":-0.3047003426,"pmnts_nmbr_din":12.8288664775,"pmnts_sum_din":30.1403120671,"loyl":97.9443976411,"pmnts_nmbr":8.3850042123,"pmnts_sum":22495.0282139848},"5":{"name":2018.0,"loyl_din":0.5428602193,"pmnts_nmbr_din":5.1019241832,"pmnts_sum_din":13.1485139268,"loyl":98.476098813

In [76]:
nmbr = df.query('year == 2022').groupby('quarter').agg({'pmnts_nmbr_per_year' : 'sum'}).reset_index()

In [77]:
s = df.query('year == 2022').groupby('quarter').agg({'pmnts_sum_per_year' : 'sum'}).reset_index()

In [78]:
churn = df.query('year == 2022').groupby('quarter').agg({'churn' : 'count'}).reset_index()

In [79]:
all = df.query('year == 2022').groupby('quarter').agg({'churn' : 'count'}).reset_index()

In [80]:
new = nmbr.merge(s.merge(all.merge(churn, on='quarter'), on='quarter') , on='quarter')

In [88]:
new

,quarter,pmnts_nmbr_per_year,pmnts_sum_per_year,loyl,loyl_din,pmnts_nmbr_din,pmnts_sum_din
0,2022Q1,16700,64385952.63,100.0,0.0,0.000000,0.000000
1,2022Q2,16766,61940823.33,100.0,0.0,0.395210,-3.797613
2,2022Q3,16688,68744704.30,100.0,0.0,-0.465227,10.984486
3,2022Q4,17061,68439028.41,100.0,0.0,2.235139,-0.444654


In [82]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [83]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [84]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [86]:
new.loyl_din = new.loyl_din.fillna(0)

In [87]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [89]:
new.rename(columns=({'quarter' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [90]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [91]:
new

,name,pmnts_nmbr,pmnts_sum,loyl,loyl_din,pmnts_nmbr_din,pmnts_sum_din
0,2022Q1,16700,64385952.63,100.0,0.0,0.000000,0.000000
1,2022Q2,16766,61940823.33,100.0,0.0,0.395210,-3.797613
2,2022Q3,16688,68744704.30,100.0,0.0,-0.465227,10.984486
3,2022Q4,17061,68439028.41,100.0,0.0,2.235139,-0.444654


In [92]:
new[new_order].T.to_json()

'{"0":{"name":"2022Q1","loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":100.0,"pmnts_nmbr":16700,"pmnts_sum":64385952.6300000027},"1":{"name":"2022Q2","loyl_din":0.0,"pmnts_nmbr_din":0.3952095808,"pmnts_sum_din":-3.7976129887,"loyl":100.0,"pmnts_nmbr":16766,"pmnts_sum":61940823.3299999982},"2":{"name":"2022Q3","loyl_din":0.0,"pmnts_nmbr_din":-0.4652272456,"pmnts_sum_din":10.9844858434,"loyl":100.0,"pmnts_nmbr":16688,"pmnts_sum":68744704.299999997},"3":{"name":"2022Q4","loyl_din":0.0,"pmnts_nmbr_din":2.2351390221,"pmnts_sum_din":-0.4446537273,"loyl":100.0,"pmnts_nmbr":17061,"pmnts_sum":68439028.4099999964}}'

In [3]:
nmbr = df.query('year == 2022').groupby('quarter').agg({'pmnts_nmbr_per_year' : 'mean'}).reset_index()

In [4]:
s = df.query('year == 2022').groupby('quarter').agg({'pmnts_sum_per_year' : 'mean'}).reset_index()

In [5]:
churn = df.query('year == 2022').groupby('quarter').agg({'churn' : 'count'}).reset_index()

In [6]:
all = df.query('year == 2022').groupby('quarter').agg({'churn' : 'count'}).reset_index()

In [7]:
new = nmbr.merge(s.merge(all.merge(churn, on='quarter'), on='quarter') , on='quarter')

In [8]:
new

,quarter,pmnts_nmbr_per_year,pmnts_sum_per_year,churn_x,churn_y
0,2022Q1,11.802120,45502.440021,1415,1415
1,2022Q2,11.659249,43074.286043,1438,1438
2,2022Q3,11.524862,47475.624517,1448,1448
3,2022Q4,11.194882,44907.498957,1524,1524


In [9]:
new['loyl'] = (new.churn_y / new.churn_x) * 100

In [10]:
new.drop(['churn_x', 'churn_y'], axis=1, inplace=True)

In [11]:
new['loyl_din'] = new['loyl'].pct_change() * 100

In [12]:
new.loyl_din = new.loyl_din.fillna(0)

In [13]:
new['pmnts_nmbr_din'] = new['pmnts_nmbr_per_year'].pct_change() * 100
new.pmnts_nmbr_din = new.pmnts_nmbr_din.fillna(0)
new['pmnts_sum_din'] = new['pmnts_sum_per_year'].pct_change() * 100
new.pmnts_sum_din = new.pmnts_sum_din.fillna(0)

In [14]:
new.rename(columns=({'quarter' : 'name', 'pmnts_nmbr_per_year' : 'pmnts_nmbr', 'pmnts_sum_per_year' : 'pmnts_sum'}), inplace=True)

In [15]:
new_order = ['name', 'loyl_din', 'pmnts_nmbr_din', 'pmnts_sum_din' , 'loyl', 'pmnts_nmbr' , 'pmnts_sum']

In [16]:
new

,name,pmnts_nmbr,pmnts_sum,loyl,loyl_din,pmnts_nmbr_din,pmnts_sum_din
0,2022Q1,11.802120,45502.440021,100.0,0.0,0.000000,0.000000
1,2022Q2,11.659249,43074.286043,100.0,0.0,-1.210555,-5.336316
2,2022Q3,11.524862,47475.624517,100.0,0.0,-1.152622,10.218018
3,2022Q4,11.194882,44907.498957,100.0,0.0,-2.863201,-5.409356


In [17]:
new[new_order].T.to_json()

'{"0":{"name":"2022Q1","loyl_din":0.0,"pmnts_nmbr_din":0.0,"pmnts_sum_din":0.0,"loyl":100.0,"pmnts_nmbr":11.8021201413,"pmnts_sum":45502.4400212014},"1":{"name":"2022Q2","loyl_din":0.0,"pmnts_nmbr_din":-1.2105552456,"pmnts_sum_din":-5.3363159799,"loyl":100.0,"pmnts_nmbr":11.6592489569,"pmnts_sum":43074.2860431154},"2":{"name":"2022Q3","loyl_din":0.0,"pmnts_nmbr_din":-1.1526220851,"pmnts_sum_din":10.2180183998,"loyl":100.0,"pmnts_nmbr":11.5248618785,"pmnts_sum":47475.6245165746},"3":{"name":"2022Q4","loyl_din":0.0,"pmnts_nmbr_din":-2.8632012441,"pmnts_sum_din":-5.4093560349,"loyl":100.0,"pmnts_nmbr":11.1948818898,"pmnts_sum":44907.4989566929}}'